In [1]:
import numpy as np 
import pandas as pd 
import os
import random
import cv2
import sys
from detectron2.data import MetadataCatalog, DatasetCatalog # read in document to know how it work
# https://detectron2.readthedocs.io/en/latest/modules/utils.html?highlight=Visualizer#detectron2.utils.visualizer.Visualizer
from detectron2.utils.visualizer import Visualizer
from utils import get_chestxray_dicts
import detectron2.data.transforms as T
from detectron2.config import get_cfg
# use default mapper 
from detectron2.data import DatasetMapper, build_detection_train_loader

In [2]:
ANNOTATIONS_CSV = "annotation_small.csv"
DETECTRON2 = "detectron2-tutorial-example"
IMG_DIR = os.path.join(DETECTRON2, "imgs")

In [3]:
df = pd.read_csv(os.path.join(DETECTRON2, ANNOTATIONS_CSV))
class_name = df.class_name.unique().tolist()
DatasetCatalog.register("my_dataset", lambda : get_chestxray_dicts(df, class_name, IMG_DIR))
MetadataCatalog.get("my_dataset").set(thing_classes = class_name)

cfg = get_cfg()
cfg.DATASETS.TRAIN = ("my_dataset")

### Generate dataloader with augmenation (Resize(256, 256))

In [4]:
dataloader = build_detection_train_loader(cfg, mapper = DatasetMapper(cfg, is_train = True, augmentations = [T.Resize((256, 256))]),
                                          total_batch_size = 5)

### Get samples from dataloader 

In [16]:
samples = iter(dataloader).__next__() # return list(dict()) with each dict() is Detectron2 format and len(samples) = total_batch_size
print(f"len(samples) = {len(samples)}")
print(f"type(samples) = {type(samples)}")

len(samples) = 5
type(samples) = <class 'list'>


### Get sample in samples

In [17]:
print(f"samples[0] = \n{samples[0]}") # return disct()

samples[0] = 
{'image_id': 44, 'file_name': 'detectron2-tutorial-example/imgs/b0a52e18d443efb28d082f0ae8e7b893.jpg', 'height': 880, 'width': 927, 'image': tensor([[[  0,   0,   0,  ...,   0,   0,   0],
         [  0,   0,   0,  ...,   0,   0,   0],
         [  0,   0,   0,  ...,   0,   0,   0],
         ...,
         [121, 131, 141,  ...,  93,  77,  63],
         [119, 131, 143,  ...,  91,  76,  63],
         [120, 131, 139,  ...,  91,  77,  63]],

        [[  0,   0,   0,  ...,   0,   0,   0],
         [  0,   0,   0,  ...,   0,   0,   0],
         [  0,   0,   0,  ...,   0,   0,   0],
         ...,
         [121, 131, 141,  ...,  93,  77,  63],
         [119, 131, 143,  ...,  91,  76,  63],
         [120, 131, 139,  ...,  91,  77,  63]],

        [[  0,   0,   0,  ...,   0,   0,   0],
         [  0,   0,   0,  ...,   0,   0,   0],
         [  0,   0,   0,  ...,   0,   0,   0],
         ...,
         [121, 131, 141,  ...,  93,  77,  63],
         [119, 131, 143,  ...,  91,  76,  63],


In [19]:
print(f"len(samples[0]) = {len(samples[0])}")
print(f"samples[0].keys() = {samples[0].keys()}")

len(samples[0]) = 6
samples[0].keys() = dict_keys(['image_id', 'file_name', 'height', 'width', 'image', 'instances'])


### Get metadata of image in sample

In [22]:
print(f"samples[0]['image'].shape = {samples[0]['image'].shape}")
print(f"samples[0]['file_name'] = {samples[0]['file_name']}")
print(f"samples[0]['image_id'] = {samples[0]['image_id']}")
print(f"samples[0]['height'] = {samples[0]['height']}")
print(f"samples[0]['width'] = {samples[0]['width']}")

samples[0]['image'].shape = torch.Size([3, 256, 256])
samples[0]['file_name'] = detectron2-tutorial-example/imgs/b0a52e18d443efb28d082f0ae8e7b893.jpg
samples[0]['image_id'] = 44
samples[0]['height'] = 880
samples[0]['width'] = 927


### Get annotations of image in sample

In [23]:
print(f"samples[0]['instances'] = {samples[0]['instances']}")

samples[0]['instances'] = Instances(num_instances=5, image_height=256, image_width=256, fields=[gt_boxes: Boxes(tensor([[ 59.0982, 148.6545, 206.8436, 206.8364],
        [125.9288,  76.5091, 161.5534, 121.3091],
        [ 61.0313, 160.2909, 179.2276, 212.9454],
        [122.8910,  74.1818, 159.8964, 122.7636],
        [ 60.2028, 147.2000, 210.4337, 203.3455]])), gt_classes: tensor([1, 0, 1, 0, 1])])


### Get fileds in instances

In [27]:
samples[0]['instances']

Instances(num_instances=5, image_height=256, image_width=256, fields=[gt_boxes: Boxes(tensor([[ 59.0982, 148.6545, 206.8436, 206.8364],
        [125.9288,  76.5091, 161.5534, 121.3091],
        [ 61.0313, 160.2909, 179.2276, 212.9454],
        [122.8910,  74.1818, 159.8964, 122.7636],
        [ 60.2028, 147.2000, 210.4337, 203.3455]])), gt_classes: tensor([1, 0, 1, 0, 1])])

In [28]:
samples[0]['instances'][:2]

Instances(num_instances=2, image_height=256, image_width=256, fields=[gt_boxes: Boxes(tensor([[ 59.0982, 148.6545, 206.8436, 206.8364],
        [125.9288,  76.5091, 161.5534, 121.3091]])), gt_classes: tensor([1, 0])])

In [30]:
print(f"samples[0]['instances']._image_size = {samples[0]['instances']._image_size}") # H, W

samples[0]['instances']._image_size = (256, 256)


In [34]:
print(f"samples[0]['instances'].get('gt_boxes') = \n{samples[0]['instances'].get('gt_boxes')}")
print()
print(f"samples[0]['instances'].get('gt_classes') = {samples[0]['instances'].get('gt_classes')}")

samples[0]['instances'].get('gt_boxes') = 
Boxes(tensor([[ 59.0982, 148.6545, 206.8436, 206.8364],
        [125.9288,  76.5091, 161.5534, 121.3091],
        [ 61.0313, 160.2909, 179.2276, 212.9454],
        [122.8910,  74.1818, 159.8964, 122.7636],
        [ 60.2028, 147.2000, 210.4337, 203.3455]]))

samples[0]['instances'].get('gt_classes') = tensor([1, 0, 1, 0, 1])


In [9]:
height, width = samples[0]['instances']._image_size
print(f"height = {height}")
print(f"width = {width}")

height = 256
width = 256


In [9]:
# compute are of each bbox
samples[0]['instances'].get("gt_boxes")

Boxes(tensor([[148.2698,  55.9328, 175.2962,  92.8115],
        [142.2639,  55.3181, 176.0469,  87.2797],
        [146.7683,  57.1621, 174.1701,  79.9040],
        [106.2287, 122.6218, 213.5836, 154.8908]]))

In [31]:
samples[0]["instances"].get("gt_boxes").tensor

tensor([[ 59.0982, 148.6545, 206.8436, 206.8364],
        [125.9288,  76.5091, 161.5534, 121.3091],
        [ 61.0313, 160.2909, 179.2276, 212.9454],
        [122.8910,  74.1818, 159.8964, 122.7636],
        [ 60.2028, 147.2000, 210.4337, 203.3455]])

In [35]:
samples[0]['instances'].get("gt_classes")

tensor([1, 0, 1, 0, 1])